In [4]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 658.6 kB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 569.5 kB/s eta 0:00:001m615.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 767.5 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 862.9 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 714.1 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 810.8 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 KB 693.4 kB/s eta 0:00:001m633.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 KB 548.4 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 572.6 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━

In [9]:
!pip3 install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 25.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 KB 51.4 MB/s eta 0:00:00


In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Attention
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split


In [51]:
data = pd.read_csv("data.csv")
data = data.dropna()
input_texts = data['brand'] + ' ' + data['title'] + ' ' + data['category'] + ' ' + data['sold_price']+' '+data['actual_price']
target_texts = data['url']

In [52]:
print("size of data: ",input_texts.shape[0])

size of data:  57422


In [53]:
input_texts

0        KOTTY Women Regular Fit Black Viscose Rayon Tr...
1        TYFFYN Women Slim High Rise Black Jeans Clothi...
2        KOTTY Women Regular Fit Black Viscose Rayon Tr...
3        mokshi Women Printed Viscose Rayon A-line Kurt...
4        LEE TEX Women Regular Fit Black Cotton Blend T...
                               ...                        
63469    Endues Printed Satin Blend Girls Multicolor Sc...
63470    NOT Boys & Girls Full Sleeve Color Block Hoode...
63471    ATXP Top - Pyjama Set Thermal For Boys & Girls...
63472    Fleximaa Boys & Girls Full Sleeve Printed Hood...
63473    LAVISH CLUB Girls Washed Denim Jacket Winter W...
Length: 57422, dtype: object

In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and preprocess data

texts = input_texts.tolist()
urls = target_texts.tolist()

# Convert labels to numerical indices
url_to_index = {url: i for i, url in enumerate(urls)}
indices = [url_to_index[url] for url in urls]

# Split data into train and test sets
train_texts, test_texts, train_indices, test_indices = train_test_split(texts, indices, test_size=0.2, random_state=42)


In [74]:
# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)


In [56]:
max_sequence_length = max(len(seq) for seq in train_sequences)
train_sequences_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

In [57]:
# Build and train the model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(urls), activation='softmax')  # Output layer with number of classes (URLs)
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_sequences_padded, np.array(train_indices), epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10


2023-08-16 21:10:37.438726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


1292/1292 [==============================] - 97s 75ms/step - loss: 11.1251 - accuracy: 0.0000e+00 - val_loss: 11.4332 - val_accuracy: 0.0000e+00
Epoch 2/10
1292/1292 [==============================] - 109s 85ms/step - loss: 10.8233 - accuracy: 0.0000e+00 - val_loss: 12.1346 - val_accuracy: 0.0000e+00
Epoch 3/10
1292/1292 [==============================] - 117s 90ms/step - loss: 10.5632 - accuracy: 4.8376e-05 - val_loss: 12.8027 - val_accuracy: 0.0000e+00
Epoch 4/10
1292/1292 [==============================] - 117s 91ms/step - loss: 9.6059 - accuracy: 2.1769e-04 - val_loss: 14.0636 - val_accuracy: 0.0000e+00
Epoch 5/10
1292/1292 [==============================] - 117s 91ms/step - loss: 8.0445 - accuracy: 0.0028 - val_loss: 16.8017 - val_accuracy: 0.0000e+00
Epoch 6/10
1292/1292 [==============================] - 117s 91ms/step - loss: 6.4338 - accuracy: 0.0211 - val_loss: 19.8838 - val_accuracy: 0.0000e+00
Epoch 7/10
1292/1292 [==============================] - 118s 91ms/step - loss: 4.

In [58]:
# Evaluate the model
loss, accuracy = model.evaluate(test_sequences_padded, np.array(test_indices))
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

359/359 [==============================] - 4s 10ms/step - loss: 30.0553 - accuracy: 0.0000e+00
Test Loss: 30.0553, Test Accuracy: 0.0000


In [1]:
# Make predictions
def predict_url(prompt):
    prompt_sequence = tokenizer.texts_to_sequences([prompt])
    prompt_sequence_padded = pad_sequences(prompt_sequence, maxlen=max_sequence_length, padding='post')
    predicted_index = np.argmax(model.predict(prompt_sequence_padded))
    predicted_url = urls[predicted_index]
    return predicted_url

# Test the chatbot
user_prompt = "Black T-shirt for men under 2000"
predicted_url = predict_url(user_prompt)
print(f"Predicted URL: {predicted_url}")

NameError: name 'tokenizer' is not defined